In [1]:
# Dependencies
import numpy as np
import pandas as pd
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [3]:
# Create our session (link) from Python to the DB
session = Session(engine)

## D1: Determine the Summary Statistics for June

In [4]:
# 1. Import the sqlalchemy extract function.
from sqlalchemy import extract

# 2. Write a query that filters the Measurement table to retrieve the temperatures for the month of June. 
results = session.query(Measurement.tobs)\
            .filter(extract('month', Measurement.date) == 6)

In [5]:
#  3. Convert the June temperatures to a list.
temperatures = results.all()

In [6]:
# 4. Create a DataFrame from the list of temperatures for the month of June. 
temperatures_df = pd.DataFrame(temperatures, columns=['June Temps'])

In [7]:
# 5. Calculate and print out the summary statistics for the June temperature DataFrame.
temperatures_df.describe()

,June Temps
count,1700.000000
mean,74.944118
std,3.257417
min,64.000000
25%,73.000000
50%,75.000000
75%,77.000000
max,85.000000


## D2: Determine the Summary Statistics for December

In [8]:
# 6. Write a query that filters the Measurement table to retrieve the temperatures for the month of December.
results = session.query(Measurement.tobs)\
            .filter(extract('month', Measurement.date) == 12)

In [9]:
# 7. Convert the December temperatures to a list.
dec_temperatures = results.all()

In [10]:
# 8. Create a DataFrame from the list of temperatures for the month of December. 
dec_temperatures_df = pd.DataFrame(dec_temperatures, columns=['December Temps'])

In [11]:
# 9. Calculate and print out the summary statistics for the Decemeber temperature DataFrame.
dec_temperatures_df.describe()

,December Temps
count,1517.000000
mean,71.041529
std,3.745920
min,56.000000
25%,69.000000
50%,71.000000
75%,74.000000
max,83.000000


## Extra Queries

In [12]:
# Query 1 - Compare the amount of preciptation in June and December.
june_precip = session.query(Measurement.prcp).filter(extract('month', Measurement.date) == 6).all()
dec_precip = session.query(Measurement.prcp).filter(extract('month', Measurement.date) == 12).all()
# Convert to DataFrames.
june_precip_df = pd.DataFrame(june_precip, columns=["June Precipitation"])
dec_precip_df = pd.DataFrame(dec_precip, columns=["December Precipitation"])

In [13]:
# Print out June statistics.
june_precip_df.describe()

,June Precipitation
count,1574.000000
mean,0.136360
std,0.335731
min,0.000000
25%,0.000000
50%,0.020000
75%,0.120000
max,4.430000


In [14]:
# Print out December statistics.
dec_precip_df.describe()

,December Precipitation
count,1405.000000
mean,0.216819
std,0.541399
min,0.000000
25%,0.000000
50%,0.030000
75%,0.150000
max,6.420000


In [15]:
# Run the query to find the average temperature by station, by month.
avg_stn_temps = session.query(Measurement.station, extract('month', Measurement.date), func.avg(Measurement.tobs))\
        .group_by(Measurement.station, extract('month', Measurement.date))\
        .order_by(extract('month', Measurement.date), Measurement.station)\
        .all()
# Convert the list to a DataFrame.
avg_stn_temps_df = pd.DataFrame(avg_stn_temps, columns=['Station', 'Month', 'Average Temp'])
# Pivot the DataFrame to use the months as the row indices and the stations as column headers.
avg_stn_temps_df.pivot(index='Month', columns='Station', values='Average Temp')

Station,USC00511918,USC00513117,USC00514830,USC00516128,USC00517948,USC00518838,USC00519281,USC00519397,USC00519523
Month,,,,,,,,,
1,66.854749,69.284553,71.453552,67.617647,68.406593,70.529412,67.923387,67.762500,70.004274
2,67.271605,69.866071,71.980645,68.309859,70.371212,70.174603,68.163717,69.534247,70.388889
3,68.565476,69.782258,71.828877,68.818182,71.363636,68.657895,68.826613,70.983193,71.266949
4,70.792899,71.579832,73.720379,70.358079,73.857143,71.235294,70.887500,74.427350,74.004367
5,72.220930,72.676113,75.248649,71.165217,75.553571,72.962963,72.000000,76.192623,75.427386
6,74.139394,74.050847,76.005376,71.937220,76.655405,73.394737,73.271186,77.559322,76.668103
7,75.238095,75.259912,77.241706,72.847826,78.039370,74.560000,74.620968,78.417355,77.862661
8,74.821429,75.723005,77.946108,73.328889,78.151786,75.860465,75.446903,78.527426,78.134199
9,74.509434,76.085106,77.469512,73.200000,78.566372,76.279070,74.823810,78.131707,77.474747


In [16]:
# FILTER BY JUNE AND DECEMBER
# Run the query to find the average temperature by station, by month.
avg_stn_temps = session.query(Measurement.station, extract('month', Measurement.date), func.avg(Measurement.tobs))\
        .filter((extract('month', Measurement.date) == 6) | (extract('month', Measurement.date) == 12))\
        .group_by(Measurement.station, extract('month', Measurement.date))\
        .order_by(extract('month', Measurement.date), Measurement.station)\
        .all()
# Convert the list to a DataFrame.
avg_stn_temps_df = pd.DataFrame(avg_stn_temps, columns=['Station', 'Month', 'Average Temp'])
# Pivot the DataFrame to use the months as the row indices and the stations as column headers.
avg_stn_temps_df.pivot(index='Month', columns='Station', values='Average Temp')

Station,USC00511918,USC00513117,USC00514830,USC00516128,USC00517948,USC00518838,USC00519281,USC00519397,USC00519523
Month,,,,,,,,,
6,74.139394,74.050847,76.005376,71.937220,76.655405,73.394737,73.271186,77.559322,76.668103
12,69.684211,71.069444,73.224719,69.291262,71.834862,72.421053,69.903226,71.109524,72.433333
